In [138]:
# pip install pandas scikit-learn matplotlib seaborn keras tensorflow xgboost lightgbm catboost

In [139]:
import pandas as pd
from sklearn import datasets
import matplotlib.pyplot as plt
import seaborn as sns
import keras
import tensorflow as tf
import xgboost as xgb
import lightgbm as lgb
import catboost as cat

# data_dict_csv = pd.read_csv(r"2025_Problem_C_Data\data_dictionary.csv")
athletes = pd.read_csv(r"2025_Problem_C_Data\summerOly_athletes.csv")
hosts = pd.read_csv(r"2025_Problem_C_Data\summerOly_hosts.csv")
medal = pd.read_csv(r"2025_Problem_C_Data\summerOly_medal_counts.csv")
programs = pd.read_csv(r"2025_Problem_C_Data\summerOly_programs.csv", encoding='windows-1252')



In [140]:
y_2028_sports = pd.read_csv(r"2025_Problem_C_Data\summerOly_2028_Sports.csv")


In [141]:
def medal_count_for_each_sport(athletes):
    train = (
        athletes[['Year', 'NOC', 'Sport', 'Event', 'Medal']].drop_duplicates().groupby(['Year', 'NOC', 'Sport', 'Medal'])
        .size()
        .unstack(fill_value=0)
        .reset_index()
    ).sort_values(by=['Year', 'Gold', 'Silver', 'Bronze'], ascending=[False, False, False, False]).loc[:, ['Year', 'NOC', 'Sport', 'Gold', 'Silver', 'Bronze', 'No medal']].reset_index(drop=True)
    return train

def process_hosts(train, hosts):
    country_to_noc = {
        "United States": "USA",
        "United Kingdom": "GBR",
        "France": "FRA",
        "Australia": "AUS",
        "Japan": "JPN",
        "Germany": "GER",
        "Greece": "GRE",
        "Brazil": "BRA",
        "China": "CHN",
        "Spain": "ESP",
        "South Korea": "KOR",
        "Russia": "RUS",
        "Canada": "CAN",
        "Mexico": "MEX",
        "Italy": "ITA",
        "Finland": "FIN",
        "Netherlands": "NED",
        "Belgium": "BEL",
        "Sweden": "SWE",
    }
    hosts['Country'] = hosts['Host'].str.split(',').str[-1].str.strip()
    hosts['NOC'] = hosts['Country'].map(country_to_noc)
    hosts.loc[hosts['Year'] == 1972, 'NOC'] = 'GER'
    hosts.loc[hosts['Year'] == 1980, 'NOC'] = 'RUS'
    hosts.loc[hosts['Year'] == 2020, 'NOC'] = 'JPN'
    hosts = hosts.drop(['Host', 'Country'], axis=1)

    train = pd.merge(train, hosts, on='Year', how='left', suffixes=('', '_host'))
    train['NOC_host'] = train.apply(lambda row: 1 if row['NOC'] == row['NOC_host'] else 0, axis=1)
    return train

def add_EWA_column(train):
    GOLD_WEIGHT = 0.5
    SILVER_WEIGHT = 0.3
    BROUNZE_WEIGHT = 0.2
    beta = 0.8

    train['current_year_value'] = (
        (GOLD_WEIGHT * train['Gold'] + SILVER_WEIGHT * train['Silver'] + BROUNZE_WEIGHT * train['Bronze']) /
        (train['Gold'] + train['Silver'] + train['Bronze'] + train['No medal'])
    )

    train = train.sort_values(by=['NOC', 'Year'])
    train['EWA'] = 0

    for noc, group in train.groupby('NOC'):
        ewa = 0
        t=1
        for idx, row in group.iterrows():
            # Update EWA based on the formula
            ewa = (beta * ewa + (1 - beta) * row['current_year_value'])/(1-beta**t)
            
            train.loc[idx, 'EWA'] = ewa
            t += 1
    train.drop(columns=['current_year_value'], inplace=True)
    return train


def combine_by_sport_name(programs):
    numeric_columns = programs.columns.difference(["Sport", "Discipline", "Code", "Sports Governing Body"])
    programs[numeric_columns] = programs[numeric_columns].apply(pd.to_numeric, errors="coerce").fillna(0).astype(int)

    aggregated_data = programs.drop(columns=["Discipline", "Code", "Sports Governing Body"]).groupby("Sport").sum()
    aggregated_data.reset_index(inplace=True)

    return aggregated_data

def add_2028_sports(programs):
    programs['Sport'] = programs['Sport'].str.replace('\xa0', ' ', regex=True)
    if '2028' not in programs.columns:
        programs = programs.merge(y_2028_sports, how='left', left_on='Sport', right_on='Sport')
        programs['2028'] = programs['2028'].fillna(0).astype(int)
    return programs

In [142]:
train = medal_count_for_each_sport(athletes)


In [143]:
train = process_hosts(train, hosts)

In [162]:
train = add_EWA_column(train)
# train[['Year','NOC','EWA']].iloc[30:35]

,Year,NOC,EWA
482,2024,AFG,0.003712
483,2024,AFG,0.002972
484,2024,AFG,0.002379
485,2024,AFG,0.001904
22802,1952,AHO,0.000000


In [164]:
programs = combine_by_sport_name(programs)
# programs[['Sport','2024']].head()

,Sport,2024
0,Aquatics,49
1,Archery,5
2,Athletics,48
3,Badminton,5
4,Baseball and Softball,0


In [146]:
programs = add_2028_sports(programs)
# programs[['Sport','2028']].head()